

This file contains a number of Python oddities, especially for someone with a Fortran/Matlab background. This list is by no means complete, but should give somone beginning with Python an idea what to look out for.

**Python variables are references**  
In Fortran/Matlab a variable can be seen as a bucket containing a object of a certain type and value. The variable and the object are one and the same thing in this approach.

In Python, a variable is always a *reference* to an object (hence we often use "name" instead of "variable"). The value of the variable is the value of the object it refers to. Objects in Python are either immutable or mutable.

**Immutable objects**  
Objects of the fundamental data-types (int, float, bool and string) are all *immutable*: changing the value of the object is not possible. Assigning a new value to a variable that refers to an immutable object has the consequence that the variable will refer to a new, different, object. This behavior is visible in the following simple example:

In [17]:
a = 1; 
b = a; 
b = 2; 
print('changing b does not change a:',a,b)
c = 2

changing b does not change a: 1 2


Very useful is the id() function (from https://docs.python.org/3/library/functions.html#id): Return the “identity” of an object. This is an integer which is guaranteed to be unique and constant for this object during its lifetime. Two objects with non-overlapping lifetimes may have the same id() value. *CPython implementation detail: This is the address of the object in memory.*

In [18]:
print('id(a) =',id(a))
print('id(b) =',id(b)) # a and b refer to different objects (1 and 2)
print('id(c) =',id(c)) # id(b)==id(c) since they both refer to 2

id(a) = 4447071784
id(b) = 4447071816
id(c) = 4447071816


A *tuple* is a container-like variable (can contain multiple, different, data-types) that is immutable:

In [19]:
a = 1, 2.0 # tuples are created using a comma (parentheses () are optional!)
print(a[0])
#a[0] = 2 # not allowed !

1


**Mutable objects**  
Python also has a other "container-like" data types (list, dict and set). At the deepest level, these will always contain objects of the fundamental (immutable) data-types, but the containers themselves are *mutable*: changing the variable can be interpreted as changing the value of the object to which the variable refers:

In [20]:
lst = [1,2,3]
print(lst,id(lst))
lst[0] = 2
print(lst,id(lst)) # value changed but ID remained the same

[1, 2, 3] 4495542464
[2, 2, 3] 4495542464


An important behavior in python is that assigning a new variable to an existing *mutable* variable only creates an alias! Thus changing the value of the object to which is being referred will be visible in both variables:

In [30]:
lst = [1,2,3]
lst2 = lst
lst[0] = 2
print(lst)
print(id(lst))
print(id(lst2)) # lst and lst2 refer to the same object

[2, 2, 3]
4495631936
4495631936


**Changing arguments of function**  
Generally, there are two ways to pass arguments to a function: by value, or by reference.  
  
*Passing by value* is easiest interpreted as making a copy of the argument as it enters the function. Modifying the variable therefore does not change the variable outside the functions. This is how Matlab function work, but note that in practice a copy is only needed if the variable is modified. Output variables need to be explicitly specified.
  
*Passing by reference* implies that the variable outside the function is also changed when the variable is modified within the function, since the object that is refered to is changed. In the "variables are buckets" interpretation this is easily understood as changing the variable in the bucket, with the bucket being passed to the function. This change will be visible also outside the function. This is how Fortran works (where additional tools are used to specify INTENT of variables.)  
  
The fact that Python variables are not buckets, but references changes things up. The important part is to known whether passed variables are mutable or not. Changing a mutable variable inside a function has the result that the variable is also changed outside the function:

In [3]:
def myfunc(a):
    a[0] = 2
    return a

a = [1,2,3]
b = myfunc(a) # both a and b are modified by the function

print(a,id(a))
print(b,id(b)) 
        

[2, 2, 3] 4463085120
[2, 2, 3] 4463085120


It is important to note that this only happens when you *change* the variable. Assigning a completely new value to the variable will *not* change the value outside the function (since this will create a new object with a new id)

In [5]:
def myfunc2(a):
    a = [4,5]
    return a

a = [1,2,3]
b = myfunc2(a) # a will not be changed since it is assigned to a new object inside the function

print(a,id(a))
print(b,id(b))

[1, 2, 3] 4462955840
[4, 5] 4462549312


Similarly, assigning a new value to immutable variables results in a new reference to a new object, this will not change the variable outside the function (outside the function, the variable "a" still refers to the original value, because nothing has changed it to refer to anything else)

In [39]:
def print_square(x):
    print(id(x)) # 
    x = x*x # x now refers to a new, different object
    print(id(x))
    print(x)

a = 2
print(id(a))
print_square(a) 
print(a) # the value of a is not changed

4447071816
4447071816
4447071880
4
2


**Changing values of iterator**  
The idea above on arguments in values carries over to Python iterators. In this case, what matters is if the objects *inside* the iterators are mutable or not:

In [10]:
a = [1,2,3] # iterator composed of immutable objects
for i in a:
    i = 4 # this does not change the inital list
print('initial list not changed:',a)

initial list not change: [1, 2, 3]


In [11]:
a = [[1],[2],[3]] # iterator composed of mutable objects
for i in a:
    i[0] = 4 # this does changes the initial list
print('initial list changed:',a)

initial list changed: [[4], [4], [4]]


Note: similarly to functions, changes to the variable only carry over outside the iteration when you *change* the variable. Assigning a completely new value to the variable will *not* change the value outside the iteration (since this will create a new object with a new id)

In [12]:
a = [[1],[2],[3]] # iterator composed of mutable objects
for i in a:
    i = 4 # this does not changes the initial list
print('initial list not changed:',a)

initial list not change: [[1], [2], [3]]


**Default values evaluated only once**

**Truth Value Testing** (from: https://docs.python.org/3/library/stdtypes.html#truth-value-testing)  
  
Any object can be tested for truth value, for use in an if or while condition or as operand of the Boolean operations below.

By default, an object is considered true unless its class defines either a __bool__() method that returns False or a __len__() method that returns zero, when called with the object. 1 Here are most of the built-in objects considered false:

* constants defined to be false: None and False  
* zero of any numeric type: 0, 0.0, 0j, Decimal(0), Fraction(0, 1)  
* empty sequences and collections: '', (), [], {}, set(), range(0)  
* Operations and built-in functions that have a Boolean result always return 0 or False for false and 1 or True for true, unless otherwise stated. (Important exception: the Boolean operations or and and always return one of their operands.)